# Mid-term Exam - Section2

# Name: Chandan Patel



### Import packages

In [2]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [3]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

# step 1: checking for existing keyspaces, if found drop it and create the new one

In [4]:
session.execute("DROP KEYSPACE IF EXISTS midterm_chandan")

In [5]:
rows = session.execute("desc keyspaces") ## chaeck the existing keyspaces and list them
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS midterm_chandan WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

In [7]:
rows = session.execute("desc keyspaces") ## checking if the keyspace created
for row in rows:
    print(f"{row[0]}")

bd23
midterm_chandan
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


# step 2: create a Song table to store song information

In [8]:
session.execute("""
CREATE TABLE IF NOT EXISTS midterm_chandan.Song ( 
    song_id INT, 
    title TEXT, 
    duration INT, 
    genre TEXT,
    PRIMARY KEY(song_id, genre)   
);
""")  ## assigned primary key as song_id and genre. Where song_id will serve as partition key (partition by song_id 's) and genre will be helpful in creating a column grouping and sorting

# step3: Loading the data and featuring the same before inserting into the table

### Load data

In [10]:
df = pd.read_csv('data.csv')
df

,song_id,title,duration,genre
0,1000,'love u like anything',250,'melody'
1,1001,'old days',200,'jazz'
2,1002,'miss u',290,'melody'
3,1003,'shaam mastani',300,'gazal'
4,1003,'Rock night',280,'rock'
5,1005,'pop songs by lucky',270,'jazz'
6,1001,'music of passion',290,'instrumental'
7,1007,'Hello world',150,'melody'


In [11]:
for index, row in df.iterrows():  ## print and check each row of the data
    print(f"song_id = {row.song_id}, title ={row.title}, duration = {row.duration}, genre = {row.genre}")

song_id = 1000, title ='love u like anything', duration = 250, genre = 'melody'
song_id = 1001, title ='old days', duration = 200, genre = 'jazz'
song_id = 1002, title ='miss u', duration = 290, genre = 'melody'
song_id = 1003, title ='shaam mastani', duration = 300, genre = 'gazal'
song_id = 1003, title ='Rock night', duration = 280, genre = 'rock'
song_id = 1005, title ='pop songs by lucky', duration = 270, genre = 'jazz'
song_id = 1001, title ='music of passion', duration = 290, genre = 'instrumental'
song_id = 1007, title ='Hello world', duration = 150, genre = 'melody'


## Step 4: Inserting data into the song table

In [12]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES ({row.song_id}, {row.title}, {row.duration}, {row.genre});
        """
    )
    session.execute(f"""
        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES ({row.song_id}, {row.title}, {row.duration}, {row.genre});
        """
    )


        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES (1000, 'love u like anything', 250, 'melody');
        

        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES (1001, 'old days', 200, 'jazz');
        

        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES (1002, 'miss u', 290, 'melody');
        

        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES (1003, 'shaam mastani', 300, 'gazal');
        

        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES (1003, 'Rock night', 280, 'rock');
        

        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES (1005, 'pop songs by lucky', 270, 'jazz');
        

        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES (1001, 'music of passion', 290, 'instrumenta

# Discussion: From the above code we have inserted all the data into the song table by iterating each row of df and inserting into the table

# step 5: Adding a record into the table

In [15]:
session.execute(f"""
        INSERT INTO midterm_chandan.song (song_id, title, duration, genre)     
        VALUES (1010,'I know you', 230, 'rock');
        """
)
rows = session.execute("select (song_id, title, duration, genre) from midterm_chandan.song")
for row in rows:
    print(f"song_id={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")

song_id=1007, title=Hello world, duration=150, genre=melody
song_id=1005, title=pop songs by lucky, duration=270, genre=jazz
song_id=1001, title=music of passion, duration=290, genre=instrumental
song_id=1001, title=old days, duration=200, genre=jazz
song_id=1003, title=shaam mastani, duration=300, genre=gazal
song_id=1003, title=Rock night, duration=280, genre=rock
song_id=1002, title=miss u, duration=290, genre=melody
song_id=1000, title=love u like anything, duration=250, genre=melody
song_id=1010, title=I know you, duration=230, genre=rock


# Discussion: Using above code we added one more row to the song table

# Step 6: Modify/Update the existing record

In [21]:

rows = session.execute("update midterm_chandan.song set title = 'best place' where song_id= 1010 and genre = 'rock'") ## update function

In [22]:
rows = session.execute("select (song_id, title, duration, genre) from midterm_chandan.song")
for row in rows:
    print(f"song_id={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")


song_id=1007, title=Hello world, duration=150, genre=melody
song_id=1005, title=pop songs by lucky, duration=270, genre=jazz
song_id=1001, title=music of passion, duration=290, genre=instrumental
song_id=1001, title=old days, duration=200, genre=jazz
song_id=1003, title=shaam mastani, duration=300, genre=gazal
song_id=1003, title=Rock night, duration=280, genre=rock
song_id=1002, title=miss u, duration=290, genre=melody
song_id=1000, title=love u like anything, duration=250, genre=melody
song_id=1010, title=best place, duration=230, genre=rock


# Discussion:  updated the existing record in the above code

## step 7: Delete the record

In [25]:
rows = session.execute("DELETE from midterm_chandan.song where song_id= 1010 and genre = 'rock'")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}")

In [27]:
rows = session.execute("select (song_id, title, duration, genre) from midterm_chandan.song")
for row in rows:
    print(f"song_id={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")
    

song_id=1007, title=Hello world, duration=150, genre=melody
song_id=1005, title=pop songs by lucky, duration=270, genre=jazz
song_id=1001, title=music of passion, duration=290, genre=instrumental
song_id=1001, title=old days, duration=200, genre=jazz
song_id=1003, title=shaam mastani, duration=300, genre=gazal
song_id=1003, title=Rock night, duration=280, genre=rock
song_id=1002, title=miss u, duration=290, genre=melody
song_id=1000, title=love u like anything, duration=250, genre=melody


# Discussion

In the above code we have demostrated all the CRUD operations for the song table, where we demonstrated the insert, update, delete functions above. By this way we can manage the keyspaces (database) and song table for the firm.  